## Сохранение и загрузка моделей: pickle и joblib

Представьте ситуацию: вы садитесь за руль автомобиля, и… вам нужно заново учиться водить! Сложная ситуация, не правда ли? В реальности мы просто заводим машину, включаем передачу и начинаем движение.

Примерно того же продакшн-среда требует от моделей. И это понятно, ведь намного дешевле с точки зрения расходования ресурсов хранить данные в виде готовой модели, чем каждый раз заново обучать модель на сервере из восьми видеокарт.

Именно поэтому код, который был написан для обучения модели и оценки её качества, крайне редко используется для инференса (от англ inference — вывод). Так называется непрерывная работа алгоритма машинного обучения в конечном приложении. По этой причине при внедрении моделей в продакшн их принято сохранять в готовом виде, то есть уже обученными и готовыми решать реальные задачи.

Изучив материалы в этом юните, вы сможете:

Подробно разобраться с тем, как с помощью библиотеки pickle сохранять обученные модели машинного обучения, загружать их обратно и заново обращаться к ним для предсказаний.
Узнать о способах передачи моделей для внедрения на языках программирования, отличных от Python.
Разобраться с темой сериализации.
Совет. Обязательно попробуйте воспроизвести примеры, которые мы будем разбирать далее.

### СЕРИАЛИЗАЦИЯ И ДЕСЕРИАЛИЗАЦИЯ

Как и почти всё в языке программирования Python, обученная модель является объектом. Этот объект не простой, поскольку модель содержит сложную иерархию классов — в каждом классе есть набор полей, ссылающихся на объекты других классов, и так далее.

Например, объект класса RandomForestClassifier из библиотеки sklearn содержит множество полей, часть из которых устанавливается во время инициализации модели (максимальная глубина, количество деревьев в ансамбле, критерий информативности и т. д.), а часть определяется во время обучения модели (последовательность предикатов внутри каждого дерева в ансамбле, значимость признаков). Наша задача — «законсервировать» этот объект (модель), сохранив значение всех полей, которые мы задали при инициализации объекта и получили по итогам обучения. То есть мы должны сохранить модель, включая её внешние и внутренние параметры.

Чтобы гарантировать сохранение всей структуры данных и получить её при загрузке обратно, используется сериализация.

Сериализация — это процесс трансформации любой структуры данных, поддерживаемой в языке, в последовательность битов (или байтов). Обратной операцией является десериализации.

ЗАЧЕМ ЭТО НУЖНО?

То, в каком виде данные записаны на диск, зачастую может сильно отличаться от того, в каком виде они существуют в памяти программы. И там, и там это будут байты информации, но в файле будет представлена просто их последовательность, а в памяти программы это может быть какой-то объект или даже несколько объектов со структурой.

Приведём простой пример с форматом CSV. Пусть у нас есть файл с одной строкой, где несколько слов записаны через запятую. На диске это будет записано просто как последовательность байтов.

Однако когда мы считываем эту последовательность внутри программы, мы хотим работать с ней как со списком строк, то есть нам нужно не только считать эти данные, но и применить к ним некоторое преобразование:

In [1]:
line = 'word1, word2, word3'

line.split(",")

## ['word1', 'word2', 'word3']

['word1', ' word2', ' word3']

Можно заметить, что после этого преобразования появилось много объектов (каждое слово списка) и некоторая структура (то, как слова расположены в этом списке). Процесс, который мы только что выполнили, называется десериализацией.

Также возможен обратный процесс, когда мы хотим сохранить в виде последовательности байтов какие-то данные из памяти программы. Например, нам нужно снова получить начальную строку:

In [2]:
",".join(['word1', 'word2', 'word3'])

## word1,word2,word3

'word1,word2,word3'

Этот процесс как раз и называется сериализацией.

Мы разобрали простейший пример, но на практике всё гораздо сложнее. Сериализовывать можно не только в текст (как с CSV, JSON и подобными форматами), но и в бинарный формат, который человек не сможет прочитать.

Бывают форматы, которые могут описывать более сложные структуры (тот же JSON). Также можно добавить сжатие итогового набора битов.

Заметим, что программа должна потратить некоторый ресурс CPU, чтобы преобразовать объект в набор байтов и наоборот.

## ИНСТРУМЕНТЫ СЕРИАЛИЗАЦИИ: PICKLE

Посмотрим, с помощью каких средств происходит сериализация объектов в Python.

В стандартную библиотеку Python входит модуль pickle, который служит для сериализации почти всех объектов произвольного типа.

Мы помним, что объекты находятся в оперативной памяти и направляются в байтовые потоки ввода-вывода. В байтовые потоки может быть направлен любой файлоподобный объект.

В ходе десериализации исходный объект воссоздаётся в оперативной памяти с теми же самыми значениями, но с новой идентичностью — новым адресом в памяти.



Обратите внимание на предупреждение в официальной документации:

⛔ Warning. The pickle module is not secure. Only unpickle data you trust.
Так как законсервирован может быть абсолютно любой объект, в нём могут быть «спрятаны» различные вредоносные программы или данные. Поэтому будьте внимательны и не проводите десериализацию бинарных файлов, в происхождении которых вы не уверены.

Не переживайте: файлы, предлагаемые для десериализации в рамках нашего курса, безопасны.

Для иллюстрации работы модуля pickle последовательно пройдёмся по всем шагам и разберёмся с тем, как сериализовать уже обученные алгоритмы машинного обучения.

ШАГ №1

Обучим модель линейной регрессии на встроенном датасете о диабете — Diabetes dataset. 

В данном датасете представлены десять исходных признаков: возраст, пол, индекс массы тела, среднее артериальное давление и шесть измерений сыворотки крови были получены для каждого из 442 пациентов с сахарным диабетом. Интерес представляет количественный показатель прогресса заболевания, замеренный через год после исходного измерения. Тип задачи — регрессия.

Примечание. Для простоты и наглядности мы опустим процесс предобработки данных, разведывательный анализ, разделение выборки на обучающую и тестовую, валидацию модели и подбор гиперпараметров, так как для наших целей это неважно. Мы уверены, что вы уже способны произвести эти этапы самостоятельно.

В качестве модели, прогнозирующей целевую переменную, возьмём простейшую линейную регрессию, и обучим её на исходных данных:

In [4]:
from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

В результате выполнения кода получился объект класса LinearRegression, на который ссылается переменная regressor. При этом атрибуты объекта (веса модели линейной регрессии) были сформированы во время обучения. То есть объект regressor теперь является обученной моделью.

ШАГ №2

Далее, когда мы получили обученную модель, нам необходимо сериализовать её, превратив объект Python в поток байтов. Для этого импортируем модуль pickle и воспользуемся функцией dumps(), в которую нужно передать объект Python.

In [5]:
import pickle

# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))
## bytes
## sklearn.linear_model._base.LinearRegression

<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


Как видим, мы создали объект model типа bytes.

ШАГ №3

Давайте попробуем восстановить (десериализовать) объект Python. Для этого в модуле pickle есть функция loads(), в которую нужно передать сериализованный объект (поток байтов).

In [6]:
# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

В результате десериализации мы смогли восстановить исходный объект (модель).

ШАГ №4

Сохраним сериализованный объект прямо в файл. Для этого в pickle есть функция dump() (без s на конце). В неё необходимо передать имя файла или ссылку на открытый файл. Файл назовём myfile, его расширение — .pkl (формат данных pickle):



In [7]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

Теперь у нас есть бинарный файл с готовой моделью, и мы можем передать его, например, ML-инженерам, которые будут заниматься деплоем модели на сервер.

ШАГ №5

Посмотрим на код, который восстанавливает (десериализует) обученную модель из файла myfile.pkl. Для этого в pickle есть функция load() (без s на конце). В неё необходимо передать имя файла или ссылку на открытый файл.

In [8]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

ШАГ №6

Убедимся, что методы и результаты предсказаний обученной модели и модели, загруженной из файла, совпадают:

In [9]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
all(regressor.predict(X) == regressor_from_bytes.predict(X))
## True
all(regressor.predict(X) == regressor_from_file.predict(X))
## True

True

Как видите, исходная и восстановленная из байтов и файла модели дают одинаковые предсказания. Это значит, что теперь мы можем импортировать наши обученные модели в любое Python-приложение и пользоваться ими, минуя этап обучения и все этапы, предшествующие ему.

ОГРАНИЧЕНИЯ

Как мы упоминали, у pickle есть ограничения. Например, мы не можем сериализовать лямбда-функции. Давайте посмотрим, что нам вернёт следующий код:

In [10]:
my_lambda = lambda x: x*2
with open('my_lambda.pkl', 'wb') as output:
    pickle.dump(my_lambda, output)
 
##"PicklingError: Can't pickle <function <lambda>"


PicklingError: Can't pickle <function <lambda> at 0x12fe5eca0>: attribute lookup <lambda> on __main__ failed

СОХРАНЕНИЕ ПАЙПЛАЙНА

Ранее мы посмотрели простейший пример сериализации готовой модели.

У вас мог возникнуть вопрос: что делать, если перед подачей данных в модель их необходимо предобработать, например произвести стандартизацию, исключить неинформативные признаки? Неужели придётся прописывать все эти шаги в коде инференса модели? А что если вопросами инференса занимаются совершенно другие специалисты, которые вообще ничего не знают о машинном обучении и не умеют производить предобработку данных?

Конечно, мы должны передать результаты в таком виде, чтобы ими можно было воспользоваться без лишних манипуляций.

Мы уже упоминали, что pickle работает с любыми объектами Python. Поэтому для сохранения может быть доступна не просто обученная модель, но и целый пайплайн, включающий предобработку данных.

Примечание. Если вы забыли, что такое пайплайны и как их формировать с помощью библиотеки sklearn, рекомендуем заглянуть в модуль ML-8. «Продвинутые методы машинного обучения».

Например, мы хотим сериализовать пайплайн, который включает в себя min-max-нормализацию и отбор пяти наиболее важных факторов на основе корреляции Пирсона. Полученные в результате данные отправляются на вход модели линейной регрессии.

In [11]:
import pickle

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x13ec879c0>)),
                ('Linear', LinearRegression())])

Пайплайн обучен. Давайте сохраним его в файл с помощью pickle:



In [12]:
# Сериализуем pipeline и записываем результат в файл
with open('my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Если сериализация завершилась успешно, то при инференсе модели мы сможем восстановить её из файла:



In [13]:
# Десериализуем pipeline из файла
with open('my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

Проверим, что результаты исходного и десериализованного пайплайнов и идентичны:



In [14]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

## True

True


Примечание. Если мы хотим сохранять сериализованные пайплайны в виде потока байтов, нужно использовать функции dumps() и loads(), а не dump() и load().

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами sklearn. Например, для решения многих задач в нашем курсе мы часто использовали feature engineering, чтобы повысить качество работы моделей. Как встроить этот шаг в исходный пайплайн?

Для этого в sklearn можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: TransformerMixin и BaseEstimator.

Посмотрим на шаблон кастомного трансформера:



In [15]:
from sklearn.base import TransformerMixin, BaseEstimator
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X

У трансформера должно быть три обязательных метода:

__init__() — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров.
Например, у трансформера для создания полиномиальных признаков PolynomialFeatures из sklearn в методе __init__() параметр degree задаёт степень полинома.
fit() — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (self).
Например, в трансформере StandardScaler в методе fit() прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода fit().
transform() — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода transform() у StandardScaler из sklearn внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе fit().

Примечание. Как мы знаем, у некоторых трансформеров из sklearn, например у того же MinMaxScaler, есть ещё и метод fit_transform(), который является комбинацией методов fit() и transform().

Наш трансформер пока что ничего не делает. Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. Давайте пропишем в методе transform() эти действия.

Для работы такого трансформера нужны только исходные данные без дополнительных параметров, поэтому методы __init__() и fit() остаются без изменений.

In [19]:
import numpy as np

In [17]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X


Посмотрим, как работает наш кастомный трансформер. Создадим объект трансформера, вызовем метод transform и посмотрим на результирующий размер таблицы.



In [20]:
# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

## Shape before transform: (442, 10)
## Shape after transform: (442, 11)

Shape before transform: (442, 10)
Shape after transform: (442, 11)


Видно, что в результате трансформации в исходную матрицу наблюдений добавился новый столбец.

Теперь давайте встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

In [21]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x13ec879c0>)),
                ('Linear', LinearRegression())])

Наконец можно сериализовать полученный pipeline:



In [22]:
# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Теперь мы можем передать пайплайн и воспользоваться им для инференса, предварительно произведя десериализацию.



In [23]:
# Десериализуем pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

Задание 2.5

Десериализуйте полученный pipeline с добавленным в него кастомной трансформации из файла. Затем предскажите значение целевой переменной для наблюдения, которое описывается следующим вектором:

In [24]:
features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])

loaded_pipe.predict(features)

array([173.01985747])

БИБЛИОТЕКА JOBLIB

Как мы видим, pickle прекрасно справляется со своей задачей: мы можем сериализовать и восстанавливать любые Python-объекты, включая модели и даже пайплайны. Однако иногда массивы данных, на которых обучаются модели, бывают настолько большими, что после загрузки из pickle невозможно восстановить объект полностью.

В таких случаях вместо pickle лучше использовать библиотеку joblib. Этот модуль более эффективен и надёжен для работы с объектами, которые содержат большие массивы данных. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой. Формат файлов для сохранения — .joblib.

В остальном работа с joblib полностью идентична работе с pickle: после обучения модели производим сериализацию с помощью функции dump(), а в коде самого приложения, где нужно использовать модель, выполняем десериализацию с помощью функции load(). В каждую из этих функций необходимо передать путь до файла для записи и чтения соответственно.

Для иллюстрации работы сохраним полученную линейную регрессию:

In [25]:
import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'regr.joblib')

## ['regr.joblib']

['regr.joblib']

Загрузим файл заново (загрузка может быть произведена в другом файле с кодом):



In [26]:
# Десериализуем модель из файла
clf_from_jobliv = joblib.load('regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

## True

True

## 3. Практика: pickle

In [52]:
with open('model.pkl', 'rb') as pkl_file:
    my_model = pickle.load(pkl_file)
    
my_model

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


/Users/egor/anaconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LinearRegression(positive=True)

In [40]:
inp = [1, 1, 1, 0.661212487096872]
my_model.predict([inp])

array([0.666])

У присланной вам модели есть два поля (атрибута) с именами a и b. Создайте из них словарь с такими же именами ключей и значениями, а затем сохраните его в файл с помощью модуля pickle.

Чтобы вы могли проверить правильность решения задания, мы создали специальный проверочный скрипт. Скачайте его здесь.

Сохраните его рядом с вашим pickle-файлом (в той же папке) и запустите, передав первым аргументом имя файла. Если вы всё сделали правильно, на экран выведется ответ для следующего задания.

КАК ЗАПУСТИТЬ СКРИПТ?

В ячейке Jupyter Notebook:

In [53]:
my_dict = {'a': my_model.a, 'b': my_model.b}
with open('my_dict.pkl', 'wb') as pkl_file:
    pickle.dump(my_dict, pkl_file)
!python hw1_check_ol.py my_dict.pkl

('secret code 2:', '3c508')


## 4. Сохранение и загрузка моделей: PMML и ONNX-ML

Среда или требования к инференсу модели для вашего проекта могут быть устроены так, что потребуют реализации на языке программирования, отличном от Python. Например, если компания разрабатывает десктопное приложение, то для внедрения модели её потребуется «перевести» на Java или C++. Как это сделать?

### PREDICTIVE MODEL MARKUP LANGUAGE

В таких случаях используется генерация файла формата PMML (Predictive Model Markup Language).

PMML — это XML-диалект, который применяется для описания статистических и DS-моделей. PMML-совместимые приложения позволяют легко обмениваться моделями данных между собой. Разработка и внедрение PMML осуществляется IT-консорциумом Data Mining Group.

Подробнее с PMML можно ознакомиться на официальном сайте.

К сожалению, далеко не все библиотеки для машинного обучения (в том числе sklearn) поддерживают возможность сохранения обученной модели в указанном формате. Однако для этого можно использовать сторонние библиотеки, и одной из самых популярных является Nyoka.

Давайте сохраним модель из предыдущего блока в формат PMML.

Для установки можно использовать систему управления пакетами pip:

In [54]:
!pip3 install nyoka


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.0/304.0 kB 7.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.8 MB/s eta 0:00:0000:0100:01


Рассмотрим пример работы с библиотекой:



In [55]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")

Итак, мы построили пайплайн обработки данных и обучили модель линейной регрессии. После этого мы с помощью функции skl_to_pmml сохранили модель в файл pipe.pmml.

Откройте файл pipe.pmml с помощью любого текстового редактора.

Давайте рассмотрим этот файл подробнее:

Секция <DataDictionary> содержит информацию о признаках, включая наименование и тип данных, используемых для построения модели.
Секция <TransformationDictionary> содержит информацию о необходимых преобразованиях для каждого признака. Обратите внимание, что в этом блоке также содержится информация для трансформации. Так как мы использовали minMaxScaler(), то в файле записаны минимальное и максимальное значения.

Таким образом, в файле содержится вся информация для того, чтобы пайплайн можно было использовать на любом языке программирования.

OPEN NEURAL NETWORK EXCHANGE

В разработке моделей на основе нейронных сетей сегодня наиболее распространён формат ONNX (Open Neural Network Exchange).

ONNX (Open Neural Network Exchange) — это открытый стандарт для обеспечения совместимости моделей машинного обучения. Он позволяет разработчикам искусственного интеллекта использовать модели с различными инфраструктурами, инструментами, средами исполнения и компиляторами.

Стандарт совместно поддерживается компаниями Microsoft, Amazon, Facebook и другими партнёрами как проект с открытым исходным кодом.

Часто стандарт ONNX и его библиотеки используют для конвертации из одного фреймворка в другой (например, из PyTorch в TensorFlow для использования в продакшене). Для конвертации различных фреймворков (не только DL) в формат ONNX и обратно существует ряд библиотек:

ONNX-Tensorflow;
Tensorflow-ONNX;
Keras-ONNX;
Sklearn-ONNX.
и другие.
Также в рамках стандарта ONNX есть инструмент ONNX-runtime. Он служит для ускорения инференса Python-моделей, а также инференса на других языках, например Java, C++.

*Задание 4.6 (со звёздочкой)

Для выполнения этого задания вам понадобится ознакомиться с документацией здесь и здесь.

В задаче ниже мы обучаем модель sklearn, конвертируем ее в ONNX и делаем инференс через ONNX-runtime.

Дополните код ниже недостающими элементами:

In [67]:
!pip install skl2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 kB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 17.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 6.1 MB/s eta 0:00:00


In [71]:
import onnxruntime as rt 
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


# загружаем данные
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(X_test)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в ONNX-формат
initial_type = [('float_input', FloatTensorType([None, 8]))]
model_onnx = convert_sklearn(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через ONNX-runtime
sess = rt.InferenceSession("model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx)

(15480, 8) (5160, 8)
sklearn model predict:
 [1.64674141 2.47861172 2.42657918 ... 1.9109084  2.03570794 2.79079465]
onnx model predict:
 [1.64674   2.478611  2.4265785 ... 1.9109077 2.0357094 2.7907944]


## 5. Деплой модели. Протоколы сетевого взаимодействия

Помните вашего коллегу Василия — того, что прислал модель в pkl-формате и просил проверить её на ваших данных? Вот, кстати, и ссылка на модель.

Завтра Василию предстоит защищать решение перед руководителями проекта, а для этого нужно показать работающий прототип (в нашем случае — задеплоить модель на тестовый сервер) и рассказать об эффективности предлагаемого решения. За деплоем Василий обратился к вам.

С чего начнём?

Во-первых, нужно разобраться с тем, как происходит взаимодействие серверов по сети.
Во-вторых, необходимо узнать, как написать сервер и обернуть в него модель. Какие есть фреймворки? Какой выбрать конкретно в нашем случае? Как его реализовать?
МОДЕЛИ СЕТЕВОГО ВЗАИМОДЕЙСТВИЯ

Начнём с первого вопроса и немного поговорим о том, как происходит взаимодействие между серверами по сети, то есть разберём процесс обмена информацией между компьютерами.

Наиболее известные модели сетевого взаимодействия — OSI и TCP/IP.

Эти модели распределяют сетевые протоколы по разным уровням взаимодействия. Что такое протокол?

Вообще протокол — это некоторый набор правил, определяющий принципы взаимодействия устройств в сети. В нашем случае это правила, по которым программа, получив по сети набор битов, понимает, как его прочитать и что он значит.

Для того чтобы обмен информацией между устройствами проходил успешно, все устройства (участники процесса) должны следовать условиям протокола. В сети поддержка протоколов встраивается или в аппаратную (в «железо»), или в программную часть (в код системы), или в обе этих части. 

На схеме ниже представлены примеры протоколов, а также уровни их распределения в модели TCP/IP:

С другими протоколами и их назначением вы можете ознакомиться здесь.

В процессе сетевого взаимодействия участвуют как минимум два устройства — устройство-отправитель и устройство-получатель. Говоря простым языком, каждая из моделей сетевых взаимодействий устанавливает правила и регламенты по отправке сообщений между компьютерами.

Отправленное сообщение проходит все уровни, начиная от прикладного уровня приложений и заканчивая физическим уровнем доступа к сети. Когда сообщение доходит до адресата, оно также проходит все уровни в обратном порядке.

Если привести аналогию из жизни, можно сказать, что обе модели описывают правила, по которым мем, который вы отправляете другу в мессенджере, будет преобразован до битов данных, передаваемых по электрическим проводам, а затем будет восстановлен по этим же битам и отображён на экране устройства вашего друга.

Нам как дата-сайентистам не нужно знать все подробности того, как работают и чем отличаются друг от друга разные уровни взаимодействия. Однако для общего развития и понимания того, как устройства обмениваются информацией, вы можете прочитать об уровнях моделей OSI и TCP/IP здесь и здесь.

Для наших целей (деплой модели в прод) достаточно уметь работать всего с тремя протоколами:

IP — протокол сетевого уровня. Он определяет путь, по которому передаются данные.

TCP — соответствует транспортному уровню, а значит, определяет, как передаются данные.

HTTP — относится к прикладному уровню, описывающему взаимодействие приложений с сетью.

IP

IP (Internet Protocol) — один из главных протоколов сетевого взаимодействия. Он отвечает за маршрутизацию трафика по сети, то есть определяет путь, по которому отправятся данные. Данные передаются пакетами (или датаграммами), которые формирует протокол IP.

Важным свойством IP является отсутствие гарантированной доставки пакетов и их цельности: пакеты могут прийти в другой очерёдности (не в той, в которой их отправляли), прийти повреждёнными (тогда они уничтожаются) или вообще не прийти.

Путь, по которому отправятся данные, строится на основе IP-адресов.

IP-адрес — это уникальный адрес, используемый для связи устройств внутри сети.

IP-адрес устроен довольно просто: чаще всего это четыре числа, разделённых точками (такой формат поддерживается в протоколе IPv4). Например, вот один из самых популярных IP-адресов — 192.168.0.1. Вы могли вводить его, чтобы зайти в настройки своего роутера.

Каждое из чисел в адресе — это восьмизначное двоичное число, или, правильнее говорить, октет. Оно может принимать значения от 0000 0000 до 1111 1111 в двоичной системе или от 0 до 255 — в десятичной системе счисления, то есть 256 разных значений.

Получается, что диапазон IP-адресов стартует с 0.0.0.0 и заканчивается 255.255.255.255. Если посчитать количество всех адресов в этом диапазоне, получится чуть больше четырёх миллиардов.

Уникальность IP-адреса может быть глобальной (в рамках всего интернета) или локальной (в рамках локальной подсети). Некоторые IP-адреса не являются общедоступными и зарезервированы для специальных целей, например диапазоны IP-адресов:

TCP

TCP (Transmission Control Protocol) — протокол транспортного уровня. Он отвечает за управление передачей данных и гарантирует:

доставку пакетов (посылает пакеты повторно, если они не были доставлены);
последовательность и целостность доставки пакетов (используя нумерацию и контрольные суммы для проверки);
устраняет дубликаты в случае необходимости.
Важной особенностью TCP является то, что перед отправкой данных он «устанавливает соединение» с получателем — обменивается управляющей информацией. После отправки пакетов источник ждёт подтверждения от получателя, что пакеты были доставлены.

Обычно на одном узле сети (сервере, компьютере) работают несколько приложений/процессов одновременно. Для идентификации приложения на источнике и получателе используется порт, который задаётся целым неотрицательным числом. Процесс или приложение могут зарезервировать у ОС определённый порт, например, для передачи данных по сети.

Порты разделяют на системные (0–1023), и пользовательские (1024–49151). Некоторые номера портов определены для конкретных приложений, например:

22 — протокол SSH для безопасной передачи данных;
25 — протокол SMTP для незащищённой передачи e-mail-сообщений;
80 — протокол HTTP.
Например, если приложение доступно по адресу 172.16.0.11:8001, это значит следующее:

172.16.0.11 — IP-адрес;
8001 — TCP-порт, отведённый приложению.

HTTP

HTTP — это наиболее широко используемый протокол. Все сайты, на которые вы заходите, работают по этому протоколу. Он был разработан именно для передачи содержимого HTML-страниц в интернете, но впоследствии стал использоваться и для других целей. Например, HTTP применяется для налаживания взаимодействия между сервисами в сложных системах. Этим он нам и интересен.

Итак, HTTP — это протокол, который работает по принципу клиент-сервер.

Это означает, что во взаимодействии участвует две программы, причём в разных ролях. Одна из них — клиент, или «заказчик услуг», формирует запрос и отправляет его к серверу. Сервер, или «поставщик услуг», получив запрос, обрабатывает его, формирует ответ и возвращает его клиенту.

1. Стартовая строка, или Request Line — по ней определяется вид запроса.
2. Заголовки запроса, или Request Headers — дополнительные параметры запроса, в которых обычно передаётся служебная информация, например, в каком формате ожидается ответ или информация о клиенте.
3. Тело запроса, или Request Message Body — содержит данные для передачи. Эта часть присутствует не всегда.

Давайте чуть подробнее разберём первую часть.

Первое, что указано в стартовой строке — это метод, или тип запроса. Есть набор стандартных методов, но теоретически вы можете создавать и свои.

Основные методы:

GET — обычно означает получение содержимого ресурса и не содержит тела.

Например, когда вы заходите в каталог интернет-магазина, вы получаете страницу с товарами — ваш браузер отправляет GET-запрос на сервер интернет-магазина.

POST — наоборот, передача данных ресурсу.

Например, когда вы заполняете форму авторизации на любом сайте и нажимаете кнопку для отправки своих данных, вы совершаете POST-запрос на сервер ресурса.

PUT — обновление ресурса.
DELETE — удаление ресурса.
В HTTP мы работаем с ресурсами, которые расположены по некоторому адресу на сервере. Изначально под ресурсами понимались HTML-файлы на сайте (вёрстка сайта), но сейчас это уже некоторое абстрактное понятие.

Адрес ресурса, или URI (Uniform Resource Identifier) — это то, что вы видите в адресной строке браузера. Он следует за методом в стартовой строке запроса.

Основное отличие — в стартовой строке: там вместо метода и URI указывается код состояния. Это численное значение, которое показывает результат обработки. Коды задаются протоколом, и вы наверняка встречали «ошибку 404» на сайтах.

404 — это как раз код состояния, означающий, что ресурса с заданным URI не существует на сервере.

Группы кодов состояния ответа HTTP-сервера делятся на следующие группы:

информационные (100–199);
успешно (200–299);
перенаправление (300–399);
ошибка клиента (400–499);
ошибка сервера (500–599).
Примеры:

Код 200 означает, что запрос обработан успешно.

Код 500 означает, что сервер не смог обработать запрос.

REST (REPRESENTATIONAL STATE TRANSFER)

img
Cпецификация HTTP не обязывает сервер понимать все методы, а также не указывает серверу, что он должен делать при получении запроса с тем или иным методом. Поэтому был изобретён архитектурный стиль REST.

Он даёт более верхнеуровневые указания, чем HTTP-протокол, а именно:

как правильно организовывать адресацию к ресурсам;
какие методы у этих ресурсов должны быть;
какой ожидается результат.

Основная концепция философии REST заключается в том, что клиентом RESTful-сервера может быть что/кто угодно: браузер, другое приложение, разработчик. Веб-приложение, спроектированное по правилам REST, предоставляет информацию о себе в форме информации о своих ресурсах.

Ресурс может быть любым объектом, о котором сервер предоставляет информацию. Например, в API Instagram ресурсом может быть пользователь, фотография, хэштег. Каждая единица информации (ресурс) однозначно определяется URL.

GET-запрос /rest/users — получение информации обо всех пользователях.
GET-запрос /rest/users/125 — получение информации о пользователе с id=125.
POST-запрос /rest/users — добавление нового пользователя.
PUT-запрос /rest/users/125 — изменение информации о пользователе с id=125.
DELETE-запрос /rest/users/125 — удаление пользователя с id=125.
Поскольку спецификация REST является общепризнанной и широко распространённой, то следовать ей очень полезно.

Если вы говорите, что ваш сервис RESTful (то есть построен по этой спецификации), то любой разработчик сразу поймёт, как примерно устроен сервис, а значит, сможет быстро начать им пользоваться: отправлять правильные запросы и получать тот результат, который ожидает.

ПРОМЕЖУТОЧНЫЕ ИТОГИ

Для того чтобы наладить взаимодействие по HTTP, на стороне клиента вам необходимо сформировать запрос: указать адрес, куда отправится запрос, выбрать метод, а также задать заголовки и тело запроса, если это нужно для выбранного метода.

На стороне сервера при получении запроса необходимо знать, какой ресурс запрошен, чтобы понять, каким кодом его обрабатывать. Этот код, в зависимости от того, что он должен делать, может проверить значения заголовков и/или прочитать тело запроса.

Отлично, со взаимодействием сервисов по сети мы разобрались. Эти теоретические знания обязательно пригодятся вам для построения собственного веб-сервиса. Теперь пора приступать к рассмотрению Python-фреймворков, которые как раз и помогут создать этот веб-сервис.

## 6. Деплой модели. Обзор фреймворков

Следующий шаг в подготовке к деплою — рассмотреть, какие существуют веб-фреймворки Python, которые позволяют реализовать серверную часть приложения и обеспечить к нему клиентский доступ.



FLASK

Flask позиционируется как микрофреймворк для написания легковесных сервисов, но при этом он достаточно гибкий и кастомизируемый, что позволяет использовать его в проектах любой сложности. Как и все остальные фреймворки, он работает на основе маршрутизации запросов, приходящих на сервер, который и должен обработать эти запросы.

В случае с Flask маршрутизация выглядит как обычные функции Python, которые вы помечаете специальным декоратором @app.route. Он связывает эту функцию с адресом, на который приходит запрос. В этой функции вы должны разобрать параметры запроса, возможно, выполнить какую-то логику и вернуть ответ клиенту.

Таким образом, для написания минимально функционального сервера достаточно добавить в скрипт всего несколько строк: необходимые импорты, создание Flask-приложения, декоратор, который свяжет адрес (endpoint, эндпоинт) с функцией, и запуск приложения. Последнее как раз и запустит работу Flask: он будет слушать запросы, маршрутизировать их и возвращать ответ.

Что такое эндпоинт?
Рассмотрим на примере. Сайт почты Mail.ru имеет адрес https://e.mail.ru. Для того чтобы попасть в папку со входящими письмами, необходимо обратиться к соответствующему интерфейсу приложения. Для этого к адресу сайта добавляется /inbox/: https://e.mail.ru/inbox. Интерфейс, который предоставляет доступ к вашим отправленным письмам, находится по адресу https://e.mail.ru/sent/. Получается, что /inbox/ и /sent/ — это и есть эндпоинты.

У Flask есть множество продвинутых функций и плагинов, написанных сторонними разработчиками для решения самых разных задач. Однако нам достаточно научиться работать с Flask на минимальном уровне, чтобы вывести свою модель как отдельный маленький сервис (микросервис), решающий одну конкретную задачу — к сервису можно обращаться по сети и получать результаты (предсказания модели).

Реализация глобальной структуры сервиса (веб-приложения) и встраивание этой структуры в микросервис — это уже задача других специалистов, например веб-разработчиков.

DJANGO

В отличие от Flask, Django нельзя назвать микрофреймворком. Напротив, даже минимальный проект для Django обычно генерируется специальными скриптами и включает с десяток файлов и папок.

Такая сложная структура нужна для громоздких приложений, чтобы не путаться в коде. Поэтому Django используют, когда нужно что-то посерьёзней, например целые сайты с базами данных.

ПРЕИМУЩЕСТВА DJANGO:

Мощный движок для рендера HTML-страниц, основанный на шаблонах.
Собственный ORM. 

ОRМ (Object Relational Mapping) — это подход, который позволяет отображать нативные объекты языка (в нашем случае классы Python) в таблицы в базе данных. Благодаря этому разработчик может абстрагироваться от написания SQL-запросов и «на лету» менять используемую базу данных. В других фреймворках вам придётся использовать сторонние ORM (самая популярная — sqlalchemy).


Огромное количество дополнительных плагинов. Один из них практически так же популярен, как и сам Django — речь о Django Rest Framework. Основываясь на ORM, он позволяет в несколько строк реализовывать эндпоинты протокола RESTful. Сам Django Rest Framework также имеет множество расширений. Благодаря этому написание сложного проекта можно свести в составлению правильной модели данных, установке и настройке подходящих расширений.

FLASK VS DJANGO

Давайте более подробно рассмотрим характеристики Flask и Django и узнаем, чем отличаются и чем похожи эти два фреймворка.

В Django есть модули для очень широкой функциональности. Чтобы получить к ним доступ, надо просто добавить нужную настройку в конфигурацию. Это отлично перекликается с девизом Django: “The web framework for perfectionists with deadlines” («Веб-фреймворк для перфекционистов, которые придерживаются сроков»).

Во Flask всё наоборот: сам фреймворк предоставляет только базовую функциональность. «Из коробки» будет доступно направление по адресам эндпойнтов, обработка запросов и ошибок, дебаггер и некоторые другие функции, явно связанные с сервером. Например, чтобы добавить авторизацию, придётся устанавливать стороннюю библиотеку. При этом вы можете выбрать ту, которая лучше подходит текущему проекту, лучше поддерживается или просто более удобна в использовании, а можете вообще не подключать библиотеки.

Flask нужен для оборачивания инференса модели в API, когда, например, по поступившему запросу запускается предикт модели. Знание Flask пригодится и на этапе создания прототипа, и на этапе тестирования.

Также некоторые компании строят весь продакшен на Flask, а значит, знать его основы будет полезно.

В связи с тем, что Django выпускается в релиз вместе со всеми сопутствующими модулями, у него достаточно длинные релиз-циклы. С Flask эта проблема менее актуальна: если какой-то функциональности не хватает, часто она есть в какой-то библиотеке. Этот подход жизнеспособен, потому что у Flask большое сообщество разработчиков, которые занимаются поддержкой библиотек для этого фреймворка.

Flask приобрёл популярность именно благодаря небольшим приложениям, для которых Django казался избыточным. Однако сегодня Flask используется не только для таких приложений.

Для справки приведём инструментарий, используемый в Django и Flask, чтобы продемонстировать, как фреймворки отличаются друг от друга по функциональности. Запоминать таблицу не нужно.

Примечание. При разработке на Flask список чаще всего не ограничивается перечисленными в таблице библиотеками — здесь собраны только наиболее популярные и известные. Также есть библиотеки, у которых есть специальные расширения для Flask. Чаще всего они не обязательны для работы с библиотекой, но могут быть полезны.


КТО ПОБЕДИЛ?

Как мы видим, для основной функциональности, реализованной в Django, есть аналоги, которые работают с Flask. Нельзя сказать, что какой-то из фреймворков проигрывает другому по большому числу параметров.

Однако стоит понимать, что, выбирая Django, вы оставляете проект со всей экосистемой Django, и что-то изменить впоследствии будет трудно. Выбирая Flask, вы получаете систему, построенную из блоков, каждый из которых проще заменить.


FASTAPI

Это относительно молодой фреймворк, однако он уже успел завоевать некоторую популярность. Возможно, через некоторое время он превратится в стандарт индустрии, заменив Flask.

Примечание. Да-да, именно Flask! FastAPI тоже позиционируется как легковесный фреймворк. Более того, принцип его использования очень похож на Flask. Авторы утверждают, что, вдохновившись Flask, они написали более быстрый (за счёт использования более современных протоколов), но при этом не уступающий по функциональности фреймворк.

Основное нововведение в FastAPI — интеграция библиотеки pydantic, которая позволяет декларативно описывать структуры запросов.

Также в FastAPI есть поддержка асинхронных функций и реализация парадигмы Dependency Injection.

img
Что такое асинхронные функции?
Асинхронная функция, или асинхронный метод — это функция, которая может быть приостановлена во время выполнения.

Это обычные функции и методы, с которым мы работали ранее. При вызове таких функций (методов) они либо выполняются до завершения, либо вызывают ошибку, либо никогда не завершаются (в функции есть бесконечный цикл).

Рассмотрим пример асинхронной функции.

Представьте, что вы когда-то поставили будильник на 8:30 и с тех пор он исправно выполняет свою работу. Чтобы понять, когда вставать, вам не нужно непрерывно смотреть на часы всю ночь. Нет нужды и посматривать на них периодически (скажем, с интервалом в пять минут). Асинхронная функция «подъём» находится в режиме ожидания. Как только произойдёт событие «на часах 8:30», она сама даст о себе знать.

Подробнее об асинхронных функциях можно прочитать здесь.

ДРУГИЕ ФРЕЙМВОРКИ

Фреймворк aiohttp тоже близок к Flask, но основан на асинхронном взаимодействии. Aiohttp добавлен в Python версии 3.4.

Tornado — относительно старый и более общий асинхронный фреймворк для написания сетевых приложений, но также он умеет работать с HTTP. По заверениям разработчиков, он очень хорошо держит нагрузку. В 2019 году Tornado занимал третье место по популярности среди разработчиков после Django и Flask.

В этом юните мы рассмотрели основные фреймворки для реализации веб-сервисов на Python. Далее в курсе мы будем использовать фреймворк Flask, однако если вы уже знакомы с другими фреймворками, вы можете использовать их.

## 7. Пишем сервер на Flask

Если бы вы попробовали написать сервер в Jupyter Notebook, то почти сразу поняли бы, что он для этого не предназначен. Как минимум потому, что после запуска вы больше не смогли бы выполнить в ноутбуке ни одной команды — программа сервера переходит в режим прослушки сети и ожидания приходящих запросов. Поэтому разработку нашего веб-сервиса мы будем вести в обычных файлах .py.

Итак, давайте напишем простое приложение на Flask.

ЧАСТЬ I. ИНИЦИАЛИЗАЦИЯ ВЕБ-ПРИЛОЖЕНИЯ

Примечание. В видео эксперт использует IDE PyCharm. Вы можете установить её самостоятельно по инструкции либо использовать ту IDE, к которой вы привыкли, например VS Code.

Сначала необходимо установить Flask. Сделать это можно с помощью pip:



In [72]:
!pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4


Теперь импортируем его и создадим объект Flask-приложения.



In [86]:
from flask import Flask
app = Flask(__name__)
@app.route('/hello')
def hello_func():
    return 'hello!'

Мы передаём __name__ при инициализации класса Flask, чтобы определить имя, с которым будет использоваться этот модуль. Flask использует расположение файла как точку, к которой он привязывает ресурсы.

Совет. В абсолютном большинстве случаев можно передавать аргумент __name__ для инициализации класса — приложение будет настроено верно.

Теперь мы можем написать функцию, которая будет обрабатывать запросы, и прикрепить её к какому-то пути (URI). Это делается с помощью специального декоратора route.

In [88]:
if __name__ == '__main__':

    app.run('localhost', 5000)

 * Serving Flask app 'server'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


Примечание. В блоке if __name__ == '__main__' прописывается код, который не должен выполняться при импорте модуля. Переменная __name__ — это специальная переменная, которая будет равна "__main__", только если файл запускается как основная программа, и выставляется равной имени модуля при импорте модуля.

Например, если мы захотим импортировать файл server.py как внешний модуль,

In [87]:
from server import *
